# Initialization

In [5]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
from arcgis.gis import *
gis = GIS()
import _params as params

c:\Users\bhereth\Anaconda3\envs\march2024\lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
dirResults = "results"

final_path = os.path.join(dirResults, "UNIONED.geojson")
print(f"📦 Loading existing unioned layer from: {final_path}")
gdfUnioned = gpd.read_file(final_path)

In [ ]:
gdfUnioned.shape[0]

1736687

In [ ]:
gdfUnioned.columns

Index(['OBJECTID', 'FID_Parcels', 'parcel_id', 'building_type', 'FID_Centers',
       'AreaName', 'AreaType', 'SHAPE_Leng', 'FID_TAZWithATOScores',
       'CO_TAZID', 'ATO_Auto_Score', 'ATO_Tran_Score',
       'FID_Interchanges_Buffers', 'dist_Interchanges',
       'FID_Interchanges_Future_Buffers', 'dist_Interchanges_Future',
       'FID_LocalBusStops_Buffers', 'dist_LocalBusStops',
       'FID_BRTStops_Buffers', 'dist_BRTStops', 'FID_BRTStops_Future_Buffers',
       'dist_BRTStops_Future', 'FID_LRTStops_Buffers', 'dist_LRTStops',
       'FID_LRTStops_Future_Buffers', 'dist_LRTStops_Future',
       'FID_CRTStops_Buffers', 'dist_CRTStops', 'FID_CRTStops_Future_Buffers',
       'dist_CRTStops_Future', 'FID_ChildCare_Buffers', 'dist_ChildCare',
       'FID_HealthCare_Buffers', 'dist_HealthCare',
       'FID_SchoolsRegPublic_Buffers', 'dist_SchoolsRegPublic',
       'FID_SchoolsHigherEd_Buffers', 'dist_SchoolsHigherEd',
       'FID_GroceryStores_Buffers', 'dist_GroceryStores',
       'FID

In [ ]:
#manually fix higher ed
gdfUnioned.loc[gdfUnioned['dist_SchoolsHigherEd']==0.5, 'dist_SchoolsHigherEd'] = 1.0

In [ ]:
gdfUnionedFiltered = gdfUnioned[(gdfUnioned['FID_Parcels']!=-1) & (gdfUnioned['FID_Communities']!=-1)]
gdfUnionedFiltered.shape[0]

1566302

In [ ]:
#sdfUnionedFiltered.spatial.to_featureclass(os.path.join(strGDB,"UNIONED_FILTERED"),sanitize_columns=False)

In [ ]:
pd.set_option('display.max_rows', None)
display(gdfUnionedFiltered.groupby(['CommCode']).agg(COUNT=('OBJECTID','size')))
pd.set_option('display.max_rows', 10)

,COUNT
CommCode,
AFK,45357
ALP,16316
BDL,7172
BTF,21721
CDF,3516
CEN,8196
CHL,10353
CLI,11560
CLR,11909


In [ ]:
#sdfParcelPartsIDsOnly = sdfUnionedFiltered[['OBJECTID','CommCode','Shape__Are', 'Shape__Len','SHAPE']]
#"ParcelPartsIDsOnly"

In [ ]:
#myFCls = "UNIONED_WEBER_FILTERED"
#arcpy.AddField_management(os.path.join(strGDB,"UNIONED_FILTERED"), "UNIQ_ID", "LONG")
#dataDict = {}
#i = 1
#updateRows = arcpy.da.UpdateCursor(os.path.join(strGDB,"UNIONED_FILTERED"), ["OBJECTID", "UNIQ_ID"])
#for updateRow in updateRows:
#    if updateRow[0] not in dataDict:
#        dataDict[updateRow[0]] = i
#        i+=1
#    updateRow[1] = dataDict[updateRow[0]]
#    updateRows.updateRow(updateRow)
#del updateRow, updateRows

# create index
gdfUnionedFiltered = gdfUnionedFiltered.reset_index()
gdfUnionedFiltered = gdfUnionedFiltered.drop(columns=('index'))
gdfUnionedFiltered = gdfUnionedFiltered.reset_index()
gdfUnionedFiltered = gdfUnionedFiltered.rename(columns={'index':'UNIQ_ID'})
gdfUnionedFiltered

,UNIQ_ID,OBJECTID,FID_Parcels,parcel_id,building_type,FID_Centers,AreaName,AreaType,SHAPE_Leng,FID_TAZWithATOScores,...,FID_1,FacilityID,SHAPE_Leng_12,FID_Communities,CommCode,NAME,COUNTYNBR,Shape__Are,Shape__Len,SHAPE
0,0,2930,13863,14766.0,Governmental,-1,,,0.000000,-1,...,0,0,0.0,29,HOO,Hooper,29,4.024017e+08,115414.297228,"{""rings"": [[[402209.3937999997, 4560107.705600..."
1,1,2966,14611,15533.0,Governmental,-1,,,0.000000,-1,...,0,0,0.0,67,PVW,Pleasant View,29,3.222566e+07,33843.751421,"{""rings"": [[[418527.93769999966, 4577296.3585]..."
2,2,3006,21393,22540.0,Agriculture,-1,,,0.000000,-1,...,0,0,0.0,29,HOO,Hooper,29,4.024017e+08,115414.297228,"{""rings"": [[[402190.6593000004, 4556506.257999..."
3,3,3144,23583,25226.0,None,-1,,,0.000000,-1,...,0,0,0.0,67,PVW,Pleasant View,29,3.222566e+07,33843.751421,"{""rings"": [[[417853.13329999987, 4578114.2476]..."
4,4,3365,53920,56523.0,No Build,-1,,,0.000000,-1,...,0,0,0.0,4,SLC,Salt Lake City,18,5.038467e+08,168892.064318,"{""rings"": [[[428847.72250000015, 4519593.5866]..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566297,1566297,1736683,575027,580297.0,Office,4,Downtown Provo,Urban Center,30147.565276,2682,...,0,0,0.0,69,PVO,Provo,25,1.966834e+08,118107.921172,"{""rings"": [[[444003.8855999997, 4454152.301000..."
1566298,1566298,1736684,575158,580430.0,Commons,4,Downtown Provo,Urban Center,30147.565276,2682,...,0,0,0.0,69,PVO,Provo,25,1.966834e+08,118107.921172,"{""rings"": [[[444003.8855999997, 4454152.301000..."
1566299,1566299,1736685,576706,582038.0,Office,4,Downtown Provo,Urban Center,30147.565276,2682,...,0,0,0.0,69,PVO,Provo,25,1.966834e+08,118107.921172,"{""rings"": [[[444003.8855999997, 4454152.301000..."
1566300,1566300,1736686,576718,582051.0,Office,4,Downtown Provo,Urban Center,30147.565276,2682,...,0,0,0.0,69,PVO,Provo,25,1.966834e+08,118107.921172,"{""rings"": [[[444003.8855999997, 4454152.301000..."


In [ ]:
# export to csv
#arcpy.conversion.TableToTable("UNIONED_WEBER_FILTERED", "E:\\GitHub\\Housing-Affordability-Weber-County\\intermediate", "unioned_weber_filtered.csv")

# DataFrame processing

In [ ]:
# Centers Scores

strCenterColName = 'CenterType'

dfCenters = pd.DataFrame({'center_code':['CM'                 ,'CU'          ,'CC'         ,'CN'                 ],
                          'center_name':['Metropolitan Center','Urban Center','City Center','Neighborhood Center']})
display(dfCenters)

for index, row in dfCenters.iterrows():
    _cname = row['center_name']
    _ccode = row['center_code']
    print(_cname)

    gdfUnionedFiltered[_ccode] = 0

    gdfUnionedFiltered.loc[(gdfUnionedFiltered[strCenterColName]==_cname),_ccode] = 1

# sum for check
display(gdfUnionedFiltered[['CM','CU','CC','CN']].sum())
display(gdfUnionedFiltered[['CM','CU','CC','CN']].max())

,center_code,center_name
0,CM,Metropolitan Center
1,CU,Urban Center
2,CC,City Center
3,CN,Neighborhood Center


Metropolitan Center
Urban Center
City Center
Neighborhood Center


CM     5638
CU    35781
CC    53615
CN    15591
dtype: int64

CM    1
CU    1
CC    1
CN    1
dtype: int64

In [ ]:
# ATO Scores
gdfUnionedFiltered['AA'] = gdfUnionedFiltered['ATO_Auto_Score']
gdfUnionedFiltered['AT'] = gdfUnionedFiltered['ATO_Tran_Score']

# sum for check
display(gdfUnionedFiltered[['AA','AT']].sum())
display(gdfUnionedFiltered[['AA','AT']].max())

AA    1.235543e+06
AT    4.732289e+05
dtype: float64

AA    1.0
AT    1.0
dtype: float64

In [ ]:
# Transportation Scores

gdfUnionedFiltered['TT'] = 0
# Transit only has single buffers
# local bus get 1/2 score, all other gets full score
gdfUnionedFiltered.loc[ (gdfUnionedFiltered['FID_LocalBusStops_Buffers'  ]!=-1) ,'TT'] = 0.5
# local bus values are overwritten by other modes
gdfUnionedFiltered.loc[((gdfUnionedFiltered['FID_BRTStops_Buffers'       ]!=-1) |
                        (gdfUnionedFiltered['FID_BRTStops_Future_Buffers']!=-1) |
                        (gdfUnionedFiltered['FID_LRTStops_Buffers'       ]!=-1) |
                        (gdfUnionedFiltered['FID_LRTStops_Future_Buffers']!=-1) |
                        (gdfUnionedFiltered['FID_CRTStops_Buffers'       ]!=-1) |
                        (gdfUnionedFiltered['FID_CRTStops_Future_Buffers']!=-1)),'TT'] = 1


# Freeway Access
dfFreewayAccess = pd.DataFrame({'tempscore' :['TF1'                                    ,'TF2'                                    ],
                                'moe'       :['Interchanges_Buffers'                   ,'Interchanges_Future_Buffers'            ],
                                'fieldname' :[getLayerDistField(strInterchangesOut_Cur),getLayerDistField(strInterchangesOut_Fut)],
                                'smallest'  :[0.5                                      ,0.5                                      ]})
                                # smallest buffer
display(dfFreewayAccess)

# future

for index, row in dfFreewayAccess.iterrows():
    gdfUnionedFiltered[row['tempscore']] = 1 / (gdfUnionedFiltered[row['fieldname']] * (1 / row['smallest'])) # equation insures smallest buffer score is always 1
    gdfUnionedFiltered[row['tempscore']].replace([np.inf, -np.inf], 0, inplace=True)

gdfUnionedFiltered['TF'] = gdfUnionedFiltered[dfFreewayAccess['tempscore'].tolist()].values.max(1)

# Active Transportation Access
dfATAccess = pd.DataFrame({'tempscore' :['TA1'                             ,'TA2'                             ,'TA3'                                     ,'TA4'                                     ],
                           'moe'       :['ATPaths_Buffers'                 ,'ATPaths_Future_Buffers'          ,'ATCycleTracks_Buffers'                   ,'ATCycleTracks_Future_Buffers'            ],
                           'fieldname' :[getLayerDistField(strPathsOut_Cur),getLayerDistField(strPathsOut_Fut),getLayerDistField(strATCycleTracksOut_Cur),getLayerDistField(strATCycleTracksOut_Fut)],
                           'smallest'  :[0.5                               ,0.5                               ,0.5                                       ,0.5                                       ]})
                            #multiplier is equal to 1 / the largest buffer distance
display(dfATAccess)

for index, row in dfATAccess.iterrows():
    gdfUnionedFiltered[row['tempscore']] = 1 / (gdfUnionedFiltered[row['fieldname']] * (1 / row['smallest'])) # equation insures smallest buffer score is always 1
    gdfUnionedFiltered[row['tempscore']].replace([np.inf, -np.inf], 0, inplace=True)

gdfUnionedFiltered['TA'] = gdfUnionedFiltered[dfATAccess['tempscore'].tolist()].values.max(1)

# sum for check
display(gdfUnionedFiltered[['TT','TF1','TF2','TF','TA']].sum())
display(gdfUnionedFiltered[['TT','TF1','TF2','TF','TA']].max())


,tempscore,moe,fieldname,smallest
0,TF1,Interchanges_Buffers,dist_Interchanges,0.5
1,TF2,Interchanges_Future_Buffers,dist_Interchanges_Future,0.5


,tempscore,moe,fieldname,smallest
0,TA1,ATPaths_Buffers,dist_ATPaths,0.5
1,TA2,ATPaths_Future_Buffers,dist_ATPaths_Future,0.5
2,TA3,ATCycleTracks_Buffers,dist_ATCycleTracks,0.5
3,TA4,ATCycleTracks_Future_Buffers,dist_ATCycleTracks_Future,0.5


TT      388296.50
TF1     275737.75
TF2      74707.25
TF      324964.50
TA     1388629.75
dtype: float64

TT     1.0
TF1    1.0
TF2    1.0
TF     1.0
TA     1.0
dtype: float64

In [ ]:
gdfUnionedFiltered

,UNIQ_ID,OBJECTID,FID_Parcels,parcel_id,building_type,FID_Centers,AreaName,AreaType,SHAPE_Leng,FID_TAZWithATOScores,...,AT,TT,TF1,TF2,TF,TA1,TA2,TA3,TA4,TA
0,0,2930,13863,14766.0,Governmental,-1,,,0.000000,-1,...,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
1,1,2966,14611,15533.0,Governmental,-1,,,0.000000,-1,...,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,2,3006,21393,22540.0,Agriculture,-1,,,0.000000,-1,...,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
3,3,3144,23583,25226.0,None,-1,,,0.000000,-1,...,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4,4,3365,53920,56523.0,No Build,-1,,,0.000000,-1,...,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566297,1566297,1736683,575027,580297.0,Office,4,Downtown Provo,Urban Center,30147.565276,2682,...,0.925136,1.0,0.25,0.0,0.25,1.0,0.0,0.0,0.0,1.0
1566298,1566298,1736684,575158,580430.0,Commons,4,Downtown Provo,Urban Center,30147.565276,2682,...,0.925136,1.0,0.25,0.0,0.25,1.0,0.0,0.0,0.0,1.0
1566299,1566299,1736685,576706,582038.0,Office,4,Downtown Provo,Urban Center,30147.565276,2682,...,0.925136,1.0,0.25,0.0,0.25,1.0,0.0,0.0,0.0,1.0
1566300,1566300,1736686,576718,582051.0,Office,4,Downtown Provo,Urban Center,30147.565276,2682,...,0.925136,1.0,0.25,0.0,0.25,1.0,0.0,0.0,0.0,1.0


# Amenities Scores

In [ ]:
#score multipliers - match to buffers above - so that max is always 1.0
#                                         Child Care                         Healthcare                          Grocery                          Community Center
dfAmenities = pd.DataFrame({'scorename' :['AC'                              ,'AH'                               ,'AG'                            ,'AM'                                    ],
                            'distfield' :[getLayerDistField(strChildCareOut),getLayerDistField(strHealthCareOut),getLayerDistField(strGroceryOut),getLayerDistField(strCommunityCenterOut)],
                            'smallest'  :[0.5                               ,0.5                                ,0.5                             ,0.5                                     ]})

display(dfAmenities)

for index, row in dfAmenities.iterrows():
    gdfUnionedFiltered[row['scorename']] = 1 / (gdfUnionedFiltered[row['distfield']] * (1 / row['smallest']))
    gdfUnionedFiltered[row['scorename']].replace([np.inf, -np.inf], 0, inplace=True)

# Education Access
dfEducation = pd.DataFrame({'tempscore' :['AE1'                                  ,'AE2'                                  ],
                            'moe'       :['SchoolsRegPublic_Buffers'             ,'SchoolsHigherEd_Buffers '             ],
                            'fieldname' :[getLayerDistField(strSchoolsOut_RegPub),getLayerDistField(strSchoolsOut_HighEd)],
                            'smallest'  :[0.5                                    ,1                                      ]})
                            # smallest buffer size

display(dfEducation)

for index, row in dfEducation.iterrows():
    gdfUnionedFiltered[row['tempscore']] = 1 / (gdfUnionedFiltered[row['fieldname']] * (1 / row['smallest'])) # equation insures smallest buffer score is always 1
    gdfUnionedFiltered[row['tempscore']].replace([np.inf, -np.inf], 0, inplace=True)

gdfUnionedFiltered['AE'] = gdfUnionedFiltered[dfEducation['tempscore'].tolist()].values.max(1)


# Parks and Open Space

gdfUnionedFiltered['AP'] = 0

gdfUnionedFiltered.loc[(gdfUnionedFiltered['FID_ParksAndOpenSpace']==1),'AP'] = 1


# sum for check
display(gdfUnionedFiltered[['AC','AH','AE1','AE2','AE','AG','AP','AM']].sum())
display(gdfUnionedFiltered[['AC','AH','AE1','AE2','AE','AG','AP','AM']].max())

,scorename,distfield,smallest
0,AC,dist_ChildCare,0.5
1,AH,dist_HealthCare,0.5
2,AG,dist_GroceryStores,0.5
3,AM,dist_CommunityCenter,0.5


,tempscore,moe,fieldname,smallest
0,AE1,SchoolsRegPublic_Buffers,dist_SchoolsRegPublic,0.5
1,AE2,SchoolsHigherEd_Buffers,dist_SchoolsHigherEd,1.0


AC     664830.50
AH     417622.00
AE1    961533.00
AE2    217462.00
AE     994876.50
AG     692023.00
AP     700328.00
AM     445898.75
dtype: float64

AC     1.0
AH     1.0
AE1    1.0
AE2    1.0
AE     1.0
AG     1.0
AP     1.0
AM     1.0
dtype: float64

# Opportunity Zone

In [ ]:
# Parks and Open Space

gdfUnionedFiltered['OZ'] = 0

gdfUnionedFiltered.loc[(gdfUnionedFiltered['FID_UtahQualifiedOpportunityZones']!=-1),'OZ'] = 1

# Building Code

In [ ]:
dBuildingCodes= [
    ["AG","Agriculture"             ],
    ["EM","Empty Buildable"         ],
    ["OS","Open Space Non-Buildable"],
    ["CH","Church"                  ],
    ["SF","Single Family Res"       ],
    ["MF","Multi Family Res"        ],
    ["GQ","Group Quarters"          ],
    ["GO","Governmental"            ],
    ["ED","Educational"             ],
    ["HE","Healthcare"              ],
    ["RE","Retail"                  ],
    ["OF","Office"                  ],
    ["IN","Industrial"              ],
    ["OT","Other"                   ],
    ["UT","Utilities"               ],
    ["NB","No Build"                ]
]

dfBuildingCodes = pd.DataFrame(dBuildingCodes, columns = ['BC','building_type'])
display(dfBuildingCodes)


gdfUnionedFilteredWithBC = pd.DataFrame.merge(gdfUnionedFiltered,dfBuildingCodes,on='building_type',how='left')
gdfUnionedFilteredWithBC['BC'] = gdfUnionedFilteredWithBC['BC'].fillna('NO')
gdfUnionedFilteredWithBC

,BC,building_type
0,AG,Agriculture
1,EM,Empty Buildable
2,OS,Open Space Non-Buildable
3,CH,Church
4,SF,Single Family Res
...,...,...
11,OF,Office
12,IN,Industrial
13,OT,Other
14,UT,Utilities


,UNIQ_ID,OBJECTID,FID_Parcels,parcel_id,building_type,FID_Centers,AreaName,AreaType,SHAPE_Leng,FID_TAZWithATOScores,...,AC,AH,AG,AM,AE1,AE2,AE,AP,OZ,BC
0,0,2930,13863,14766.0,Governmental,-1,,,0.000000,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,GO
1,1,2966,14611,15533.0,Governmental,-1,,,0.000000,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,GO
2,2,3006,21393,22540.0,Agriculture,-1,,,0.000000,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,AG
3,3,3144,23583,25226.0,None,-1,,,0.000000,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,NO
4,4,3365,53920,56523.0,No Build,-1,,,0.000000,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,NB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566297,1566297,1736683,575027,580297.0,Office,4,Downtown Provo,Urban Center,30147.565276,2682,...,1.0,1.0,1.0,1.0,0.5,0.5,0.5,0,1,OF
1566298,1566298,1736684,575158,580430.0,Commons,4,Downtown Provo,Urban Center,30147.565276,2682,...,1.0,1.0,1.0,1.0,0.5,0.5,0.5,0,1,NO
1566299,1566299,1736685,576706,582038.0,Office,4,Downtown Provo,Urban Center,30147.565276,2682,...,1.0,1.0,1.0,1.0,0.5,0.5,0.5,0,1,OF
1566300,1566300,1736686,576718,582051.0,Office,4,Downtown Provo,Urban Center,30147.565276,2682,...,1.0,1.0,1.0,1.0,0.5,0.5,0.5,0,1,OF


In [ ]:
gdfUnionedFilteredWithBC.columns

Index(['UNIQ_ID', 'OBJECTID', 'FID_Parcels', 'parcel_id', 'building_type',
       'FID_Centers', 'AreaName', 'AreaType', 'SHAPE_Leng',
       'FID_TAZWithATOScores', 'CO_TAZID', 'ATO_Auto_Score', 'ATO_Tran_Score',
       'FID_Interchanges_Buffers', 'dist_Interchanges',
       'FID_Interchanges_Future_Buffers', 'dist_Interchanges_Future',
       'FID_LocalBusStops_Buffers', 'dist_LocalBusStops',
       'FID_BRTStops_Buffers', 'dist_BRTStops', 'FID_BRTStops_Future_Buffers',
       'dist_BRTStops_Future', 'FID_LRTStops_Buffers', 'dist_LRTStops',
       'FID_LRTStops_Future_Buffers', 'dist_LRTStops_Future',
       'FID_CRTStops_Buffers', 'dist_CRTStops', 'FID_CRTStops_Future_Buffers',
       'dist_CRTStops_Future', 'FID_ChildCare_Buffers', 'dist_ChildCare',
       'FID_HealthCare_Buffers', 'dist_HealthCare',
       'FID_SchoolsRegPublic_Buffers', 'dist_SchoolsRegPublic',
       'FID_SchoolsHigherEd_Buffers', 'dist_SchoolsHigherEd',
       'FID_GroceryStores_Buffers', 'dist_GroceryStores',


In [ ]:
gdfUnionedFiltered_Scores = gdfUnionedFilteredWithBC[['OBJECTID','UNIQ_ID','CommCode','COUNTYNBR','BC','OZ','CM','CU','CC','CN','AA','AT','TT','TF','TA','AC','AH','AE','AG','AP','AM','SHAPE','Shape__Are','Shape__Len']]
display(gdfUnionedFiltered_Scores)
display(gdfUnionedFiltered_Scores.describe())
#display(gdfUnionedFiltered_Scores.sum())

,OBJECTID,UNIQ_ID,CommCode,COUNTYNBR,BC,OZ,CM,CU,CC,CN,...,TA,AC,AH,AE,AG,AP,AM,SHAPE,Shape__Are,Shape__Len
0,2930,0,HOO,29,GO,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,"{'rings': [[[402209.3937999997, 4560107.705600...",4.024017e+08,115414.297228
1,2966,1,PVW,29,GO,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,"{'rings': [[[418527.93769999966, 4577296.3585]...",3.222566e+07,33843.751421
2,3006,2,HOO,29,AG,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,"{'rings': [[[402190.6593000004, 4556506.257999...",4.024017e+08,115414.297228
3,3144,3,PVW,29,NO,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,"{'rings': [[[417853.13329999987, 4578114.2476]...",3.222566e+07,33843.751421
4,3365,4,SLC,18,NB,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,"{'rings': [[[428847.72250000015, 4519593.5866]...",5.038467e+08,168892.064318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566297,1736683,1566297,PVO,25,OF,1,0,1,0,0,...,1.0,1.0,1.0,0.5,1.0,0,1.0,"{'rings': [[[444003.8855999997, 4454152.301000...",1.966834e+08,118107.921172
1566298,1736684,1566298,PVO,25,NO,1,0,1,0,0,...,1.0,1.0,1.0,0.5,1.0,0,1.0,"{'rings': [[[444003.8855999997, 4454152.301000...",1.966834e+08,118107.921172
1566299,1736685,1566299,PVO,25,OF,1,0,1,0,0,...,1.0,1.0,1.0,0.5,1.0,0,1.0,"{'rings': [[[444003.8855999997, 4454152.301000...",1.966834e+08,118107.921172
1566300,1736686,1566300,PVO,25,OF,1,0,1,0,0,...,1.0,1.0,1.0,0.5,1.0,0,1.0,"{'rings': [[[444003.8855999997, 4454152.301000...",1.966834e+08,118107.921172


,OBJECTID,UNIQ_ID,OZ,CM,CU,CC,CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM,Shape__Are,Shape__Len
count,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06,1.566302e+06
mean,9.007182e+05,7.831505e+05,4.983905e-02,3.599561e-03,2.284425e-02,3.423031e-02,9.954019e-03,7.888283e-01,3.021313e-01,2.479065e-01,2.074724e-01,8.865658e-01,4.244587e-01,2.666293e-01,6.351754e-01,4.418196e-01,4.471219e-01,2.846825e-01,1.170435e+08,7.860440e+04
std,4.829830e+05,4.521526e+05,2.176124e-01,5.988829e-02,1.494069e-01,1.818203e-01,9.927206e-02,1.549708e-01,2.825385e-01,3.535876e-01,2.595530e-01,2.295215e-01,4.041859e-01,3.594077e-01,3.656647e-01,3.965116e-01,4.971962e-01,2.877280e-01,1.166516e+08,4.070559e+04
min,2.930000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.407033e+06,8.080142e+03
25%,4.867372e+05,3.915752e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.227040e-01,8.182118e-03,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,4.654612e+07,4.510288e+04
50%,8.989995e+05,7.831505e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.220489e-01,2.471518e-01,0.000000e+00,2.500000e-01,1.000000e+00,5.000000e-01,0.000000e+00,5.000000e-01,5.000000e-01,0.000000e+00,2.500000e-01,8.310520e+07,6.967324e+04
75%,1.313913e+06,1.174726e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.974273e-01,4.918644e-01,5.000000e-01,2.500000e-01,1.000000e+00,1.000000e+00,5.000000e-01,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e-01,1.301002e+08,1.026423e+05
max,1.736687e+06,1.566301e+06,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.038467e+08,1.688921e+05


In [ ]:
# Reproject to WGS84 if needed for export
gdfUnionedFiltered_Scores = gdfUnionedFiltered_Scores.to_crs(epsg=4326)

# Export to GeoJSON (or change driver to "ESRI Shapefile" if required)
output_path = os.path.join(dirResults, "ParcelPartsWithScores.geojson")
gdfUnionedFiltered_Scores.to_file(output_path, driver="GeoJSON")

print(f"✅ Exported ParcelPartsWithScores to {output_path}")


dissolve_fields = ['CommCode','COUNTYNBR','BC','OZ','CM','CU','CC','CN','AA',
                   'AT','TT','TF','TA','AC','AH','AE','AG','AP','AM']

# Dissolve and aggregate geometries
gdfDissolved = gdfUnionedFiltered_Scores.dissolve(by=dissolve_fields, as_index=False)

# Reproject and export
gdfDissolved = gdfDissolved.to_crs(epsg=4326)
output_dissolved = os.path.join(dirResults, "ParcelPartsWithScores_Dissolved.geojson")
gdfDissolved.to_file(output_dissolved, driver="GeoJSON")

print(f"✅ Exported dissolved ParcelPartsWithScores to {output_dissolved}")


c:\Users\bhereth\AppData\Local\ESRI\conda\envs\arcgispro-py3-billenv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


'E:\\GitHub\\Housing-Suitability\\results\\HousingSuitability.gdb\\ParcelPiecesWithScores'

In [ ]:
# delete layers that aren't needed anymore
deleteIfExists(strInterchangesOut_Cur  + "_Buffers")
deleteIfExists(strInterchangesOut_Fut  + "_Buffers")
deleteIfExists(strTransitOut_Lcl       + "_Buffers")
deleteIfExists(strTransitOut_BrtCur    + "_Buffers")
deleteIfExists(strTransitOut_BrtFut    + "_Buffers")
deleteIfExists(strTransitOut_LrtCur    + "_Buffers")
deleteIfExists(strTransitOut_LrtFut    + "_Buffers")
deleteIfExists(strTransitOut_CrtCur    + "_Buffers")
deleteIfExists(strTransitOut_CrtFut    + "_Buffers")
deleteIfExists(strChildCareOut         + "_Buffers")
deleteIfExists(strHealthCareOut        + "_Buffers")
deleteIfExists(strSchoolsOut_RegPub    + "_Buffers")
deleteIfExists(strSchoolsOut_HighEd    + "_Buffers")
deleteIfExists(strGroceryOut           + "_Buffers")
deleteIfExists(strCommunityCenterOut   + "_Buffers")
deleteIfExists(strPathsOut_Cur         + "_Buffers")
deleteIfExists(strPathsOut_Fut         + "_Buffers")
deleteIfExists(strATCycleTracksOut_Cur + "_Buffers")
deleteIfExists(strATCycleTracksOut_Fut + "_Buffers")

In [ ]:
dfCommCodeCat = gdfUnionedFiltered_Scores.groupby(['CommCode'],as_index=False).agg(OZ=('OZ','max'),
                                                                                   CM=('CM','max'),
                                                                                   CU=('CU','max'),
                                                                                   CC=('CC','max'),
                                                                                   CN=('CN','max'),
                                                                                   AA=('AA','max'),
                                                                                   AT=('AT','max'),
                                                                                   TT=('TT','max'),
                                                                                   TF=('TF','max'),
                                                                                   TA=('TA','max'),
                                                                                   AC=('AC','max'),
                                                                                   AH=('AH','max'),
                                                                                   AE=('AE','max'),
                                                                                   AG=('AG','max'),
                                                                                   AP=('AP','max'),
                                                                                   AM=('AM','max'))

dfCommCodeCat_melt = pd.melt(dfCommCodeCat, id_vars='CommCode', value_vars=['OZ','CM','CU','CC','CN','AA','AT','TT','TF','TA','AC','AH','AE','AG','AP','AM'], var_name='Cat', value_name='CatValue', col_level=None)

dfCommCodeCatWithValues = dfCommCodeCat_melt[dfCommCodeCat_melt['CatValue']>0].copy()
dfCommCodeCatWithValues.drop(columns=['CatValue'],inplace=True)
dfCommCodeValidCategories = dfCommCodeCatWithValues[~dfCommCodeCatWithValues['Cat'].str.startswith('OZ')]
dfCommCodeValidCategories = dfCommCodeCatWithValues.groupby(['CommCode'],as_index=False).agg(['unique'])
dfCommCodeValidCategories = dfCommCodeValidCategories.droplevel(level=0, axis=1)
dfCommCodeValidCategories = dfCommCodeValidCategories.reset_index()
dfCommCodeValidCategories = dfCommCodeValidCategories.rename(columns={'unique':'categories'})
dfCommCodeValidCategories['categories'] = [','.join(map(str, l)) for l in dfCommCodeValidCategories['categories']]
dfCommCodeValidCategories


,CommCode,categories
0,AFK,"OZ,CC,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM"
1,ALP,"AA,AT,TA,AC,AH,AE,AG,AP,AM"
2,BDL,"CU,CC,CN,AA,AT,TT,TF,TA,AC,AE,AG,AP,AM"
3,BTF,"CC,CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM"
4,CDF,"AA,TA,AE,AP"
...,...,...
68,WJC,"OZ,CC,CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM"
69,WMT,"CN,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM"
70,WPT,"CC,CN,AA,AT,TT,TA,AC,AH,AE,AG,AP"
71,WTR,"CC,AA,AT,TT,TF,TA,AC,AH,AE,AG,AP,AM"


In [ ]:
dfCommCodeValidCenters = dfCommCodeCat_melt[dfCommCodeCat_melt['CatValue']>0].copy()
dfCommCodeValidCenters.drop(columns=['CatValue'],inplace=True)
dfCommCodeValidCenters = dfCommCodeValidCenters[dfCommCodeValidCenters['Cat'].str.startswith('C')]
dfCommCodeValidCenters = dfCommCodeValidCenters.groupby(['CommCode'],as_index=False).agg(['unique'])
dfCommCodeValidCenters = dfCommCodeValidCenters.droplevel(level=0, axis=1)
dfCommCodeValidCenters = dfCommCodeValidCenters.reset_index()
dfCommCodeValidCenters = dfCommCodeValidCenters.rename(columns={'unique':'centers'})
dfCommCodeValidCenters['centers'] = [','.join(map(str, l)) for l in dfCommCodeValidCenters['centers']]
dfCommCodeValidCenters

,CommCode,centers
0,AFK,CC
1,BDL,"CU,CC,CN"
2,BTF,"CC,CN"
3,CEN,"CC,CN"
4,CLI,CN
...,...,...
53,WJC,"CC,CN"
54,WMT,CN
55,WPT,"CC,CN"
56,WTR,CC


In [ ]:
dfCommCodeCatWithOZ = gdfUnionedFiltered_Scores.groupby(['CommCode'],as_index=False).agg(OZ=('OZ','max'))
dfCommCodeCatWithOZ
dfCommCodeCatWithOZ.loc[dfCommCodeCatWithOZ['OZ']>0,'oppzones']='yes'
dfCommCodeCatWithOZ.loc[dfCommCodeCatWithOZ['OZ']==0,'oppzones']='no'
dfCommCodeCatWithOZ.drop(columns=['OZ'],inplace=True)
dfCommCodeCatWithOZ

,CommCode,oppzones
0,AFK,yes
1,ALP,no
2,BDL,no
3,BTF,no
4,CDF,no
...,...,...
68,WJC,yes
69,WMT,no
70,WPT,no
71,WTR,no


In [ ]:
dfCommunitiesForApp = sdfCommunitiesWithCommCode.copy()
dfCommunitiesForApp = dfCommunitiesForApp[['NAME','CommCode','COUNTYNBR']]
dfCommunitiesForApp = pd.DataFrame.merge(dfCommunitiesForApp,dfCommCodeCatWithOZ      ,on='CommCode',how='left')
dfCommunitiesForApp = pd.DataFrame.merge(dfCommunitiesForApp,dfCommCodeValidCenters   ,on='CommCode',how='left')
dfCommunitiesForApp = pd.DataFrame.merge(dfCommunitiesForApp,dfCommCodeValidCategories,on='CommCode',how='left')
dfCommunitiesForApp.rename(columns={'CommCode':'value','NAME':'label'},inplace=True)
dfCommunitiesForApp = dfCommunitiesForApp.sort_values(by=['label'])
dfCommunitiesForApp.to_json('communities.json',orient='records')
dfCommunitiesForApp

NameError: name 'sdfCommunitiesWithCommCode' is not defined